<a href="https://colab.research.google.com/github/MyPham1207/Python_for_ML/blob/main/Assignments/Classification/Classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

import numpy as np

Mounted at /content/drive


##Step 1: Load data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/School/Python_for_ML/Assignment_6/Data/Wine.csv')
# shuffle the data or else the data after split will be skewed
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,12.60,1.34,1.90,18.5,88,1.45,1.36,0.29,1.35,2.45,1.04,2.77,562,2
1,13.86,1.51,2.67,25.0,86,2.95,2.86,0.21,1.87,3.38,1.36,3.16,410,2
2,13.07,1.50,2.10,15.5,98,2.40,2.64,0.28,1.37,3.70,1.18,2.69,1020,1
3,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450,1
4,11.66,1.88,1.92,16.0,97,1.61,1.57,0.34,1.15,3.80,1.23,2.14,428,2


##Step 2: Split the data and standardization

In [ ]:
# Split the data
n_splits = 5
kf = KFold(n_splits)
train_index = []
test_index = []
for train, test in kf.split(data):
  train_index.append(train)
  test_index.append(test)

In [ ]:
train_data = []
test_data = []
for i in range(n_splits):
  train_data.append(data.iloc[train_index[i]])
  test_data.append(data.iloc[test_index[i]])

train_data[0].head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
36,13.49,3.59,2.19,19.5,88,1.62,0.48,0.58,0.88,5.70,0.81,1.82,580,3
37,11.61,1.35,2.70,20.0,94,2.74,2.92,0.29,2.49,2.65,0.96,3.26,680,2
38,13.51,1.80,2.65,19.0,110,2.35,2.53,0.29,1.54,4.20,1.10,2.87,1095,1
39,13.64,3.10,2.56,15.2,116,2.70,3.03,0.17,1.66,5.10,0.96,3.36,845,1
40,12.08,1.83,2.32,18.5,81,1.60,1.50,0.52,1.64,2.40,1.08,2.27,480,2


In [ ]:
norm_train_data = []
norm_test_data = []
for i in range(n_splits): 
  scaler = StandardScaler()
  transformer = scaler.fit(train_data[i])
  train_temp = pd.DataFrame(transformer.transform(train_data[i]), columns=train_data[i].columns, index=train_data[i].index) 
  test_temp = pd.DataFrame(transformer.transform(test_data[i]), columns=test_data[i].columns, index=test_data[i].index)
  norm_train_data.append(train_temp)
  norm_test_data.append(train_temp)


##Step 4: Training and calculating accuracy (have PCA)

In [ ]:
col = list(data.columns)
X_tag = col[0:len(col) - 1]
y_tag = [col[len(col) - 1]]
y_tag

['Customer_Segment']

In [ ]:
def train_and_predict(norm_train_data, norm_test_data, n_splits, model, use_PCA=False, num_components=2):
  y_predict = []
  for i in range(n_splits):

    if use_PCA:
      pca = PCA(n_components=num_components)
      pca = pca.fit(norm_train_data[i][X_tag])
      X_train = pca.transform(norm_train_data[i][X_tag])
      X_test = pca.transform(norm_test_data[i][X_tag])
    else:
      X_train = norm_train_data[i][X_tag]
      X_test = norm_test_data[i][X_tag]

    y_train = np.ravel(norm_train_data[i][y_tag])
    y_train_round = [round(y,0) for y in y_train]

    y_pred = model.fit(X_train, y_train_round).predict(X_test)
    y_predict.append(y_pred)

  return y_predict

In [ ]:
def accuracyCalc(y_predict, norm_test_data, n_splits):
  accuracy = 0
  for i in range(n_splits):
    
    y_test = np.ravel(norm_test_data[i][y_tag])
    y_test_round = [round(y,0) for y in y_test]
  
    acc = accuracy_score(y_test_round,  y_predict[i])
    accuracy += acc
  accuracy = accuracy / n_splits * 100
  return accuracy

In [ ]:
out = open('/content/drive/MyDrive/School/Python_for_ML/Assignment_6/Data/Accuracy_with_and_without_PCA.txt', 'w')

##a.Gaussian Naive Bayes

In [ ]:
model = GaussianNB()
out.write('- Gaussian Naive Bayes: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##b.Logistic Regression

In [ ]:
model = LogisticRegression(random_state=0)
out.write('- Logistic Regression: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##c.K-Nearest Neighbors

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
out.write('- K-Nearest Neighbors: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##d.Support Vector Machine

In [ ]:
model = svm.SVC()
out.write('- Support Vector Machine: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##e.Decision Tree

In [ ]:
model = tree.DecisionTreeClassifier()
out.write('- Decision Tree: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

##f.Random Forest

In [ ]:
model = RandomForestClassifier(max_depth=2, random_state=0)
out.write('- Random Forest: \n')
y_predict1 = train_and_predict(norm_train_data, norm_test_data, n_splits, model)
acc1 = accuracyCalc(y_predict1, norm_test_data, n_splits)
out.write('+ Accuracy without PCA: ' + str(acc1) + '\n')
y_predict2 = train_and_predict(norm_train_data, norm_test_data, n_splits, model, True, 10)
acc2 = accuracyCalc(y_predict2, norm_test_data, n_splits)
out.write('+ Accuracy with PCA: ' + str(acc2) + '\n')
out.write('\n')

1

In [ ]:
out.close()